In [57]:
import pandas  as pd
import numpy as np

from sklearn.feature_selection import VarianceThreshold

In [58]:
relevant_genes = pd.read_csv("DaniDatasets/relevant_genes.csv")

In [59]:
microbiome = pd.read_csv("DaniDatasets/BLCA_csv/microbiome_log_rna.csv",index_col=0)
mirna = pd.read_csv("DaniDatasets/BLCA_csv/miRNA.csv",index_col=0)
svs = pd.read_csv("DaniDatasets/BLCA_csv/svs.csv", index_col=0)
mutations = pd.read_csv("DaniDatasets/BLCA_csv/mutations.csv",index_col=0)
rna_median = pd.read_csv("DaniDatasets/BLCA_csv/RNA_expression_median.csv", index_col=0)
rppa_levels = pd.read_csv("DaniDatasets/BLCA_csv/rppa_levels.csv", index_col=0)
metilation = pd.read_csv("DaniDatasets/BLCA_csv/methylations.csv", index_col=0)
cna_log2 = pd.read_csv("DaniDatasets/BLCA_csv/cna_log2.csv", index_col=0)

In [60]:
for omic in [mutations, metilation, rna_median, cna_log2]:
    columns_to_drop = []
    for column_name in mutations.columns:
        if not (relevant_genes['Gene'] == column_name).any():
            columns_to_drop.append(column_name)
    print("dropped {} columns".format(len(columns_to_drop)))
    omic.drop(columns=columns_to_drop, inplace=True)
    

dropped 0 columns
dropped 0 columns
dropped 0 columns
dropped 0 columns


In [61]:
## svs has a particular configuration
columns_to_drop = []
for column_name in svs.columns:
    gene1, gene2 = column_name.split('_')
    # Check if either gene is in genes_df
    if not ((relevant_genes['Gene'] == gene1).any() 
            or (relevant_genes['Gene'] == gene2).any()):
        columns_to_drop.append(column_name)

# Dropping
svs.drop(columns=columns_to_drop, inplace=True)
svs.shape

(332, 377)

In [62]:
# check for index naming
for i in range(0,len(svs.index)):
    if svs.index[i] != mutations.index[i]:
        print("Concatenation not Correct")
alterations = pd.concat([svs, mutations], axis=1)

In [63]:
# set unique column name
for omic, name in zip([rna_median, metilation, mirna, cna_log2, rppa_levels, alterations, microbiome],
                      ['rna_median', 'metilation', 'mirna', 'cna_log2', 'rppa_levels', 'alterations', 'microbiome']):
    omic.columns = [f"{name}_{c}" for c in omic.columns]

In [64]:
clinical_total = pd.read_csv("DaniDatasets/BLCA_csv/clinical_data.csv", index_col=1)

In [65]:
clinical = clinical_total[['AGE', 'GENDER', 'OS_STATUS', 'OS_MONTHS']]

In [66]:
def convert_status(status):
    if status == '1:DECEASED':
        return 1
    elif status == '0:LIVING':
        return 0
    else:
        print("ERROR: y cannot be empty")
        return np.NaN

def convert_gender(gender):
    if gender == 'Female':
        return 1
    elif gender == 'Male':
        return 0
    else:
        print("ERROR: y cannot be empty in gender")
        return np.NaN


clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])


clinical['GENDER'] = clinical['GENDER'].apply(convert_gender)
clinical['GENDER'] = pd.to_numeric(clinical['GENDER'])

/tmp/ipykernel_3059/2970686832.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
/tmp/ipykernel_3059/2970686832.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])
/tmp/ipykernel_3059/2970686832.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [67]:
# remove patients wit more than 80 yo and less than 6 months OS
dropped = 0
dropped += len(clinical[((clinical['AGE'] >= 80) & 
                         (clinical['OS_MONTHS'] <= 6))])

dropped += len(clinical[clinical['OS_MONTHS']
               .isna() == True])

filtered_clinical = clinical[~(
    (clinical['AGE'] >= 80) & 
    (clinical['OS_MONTHS'] <= 6))]

filtered_clinical = filtered_clinical[filtered_clinical['OS_MONTHS'].isna() == False]

print("dropped {} rows".format(dropped))

dropped 13 rows


In [68]:
print(microbiome.shape, filtered_clinical.shape)

(332, 1406) (319, 4)


In [69]:
## create dictionary of DF
dataframes = [filtered_clinical, rna_median, metilation, mirna, cna_log2, rppa_levels, alterations, microbiome]
d = {}
for k, v in zip(dataframes,
        ["filtered_clinical", "rna_median", "metilation", "mirna", "cna_log2", "rppa_levels", "alterations", "microbiome"]):
    d[v] = k

In [93]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.01 ) # 0.0001 at first step
## apparently now complains about 0.0099 variance too -> consider going to 0.01

dropped_columns = 0

for k in d.keys():
    before = d[k].shape[1]
    transformed_data = selector.fit_transform(d[k])
    selected_features_mask = selector.get_support() # support mask
    
    d[k] = pd.DataFrame(transformed_data, 
                        index = d[k].index, 
                        columns = d[k].columns[selected_features_mask])
    
    after = d[k].shape[1]
    dropped = before - after
    print(f"Dropped {dropped} columns in omic: {k}")  
    dropped_columns += dropped
    
print(f"Total dropped columns due to low variance: {dropped_columns}")

Dropped 0 columns in omic: filtered_clinical
Dropped 6 columns in omic: rna_median
Dropped 770 columns in omic: metilation
Dropped 37 columns in omic: mirna
Dropped 0 columns in omic: cna_log2
Dropped 66 columns in omic: rppa_levels
Dropped 854 columns in omic: alterations
Dropped 63 columns in omic: microbiome
Total dropped columns due to low variance: 1796


In [95]:
# dropped_columns = 0

# for k in d.keys():
#     before = d[k].shape[1]
#     to_keep = [col for col in d[k].columns if len(d[k][col].unique()) > 1]
#     d[k] = d[k][to_keep]
#     after = d[k].shape[1]
    
#     dropped = before - after
#     print(f"Dropped {dropped} columns in omic: {k}")  
#     dropped_columns += dropped
    
# print(f"Total dropped columns due to low variance: {dropped_columns}")


In [96]:
type(d[k])

pandas.core.frame.DataFrame

In [97]:
def merge_all(omics):
    df = omics[0]
    starting_indices = [0,omics[0].shape[1]]
    for omic in omics[1:]:
        #print(omic)
        starting_indices.append(starting_indices[-1] + omic.shape[1])
        df = df.merge(omic, left_index=True, right_index=True)
    return (df, starting_indices)

In [98]:
final_df, indices = merge_all(
    [d['filtered_clinical'], 
     d['rna_median'], 
     d['metilation'], 
     d['mirna' ],
     d['cna_log2'], 
     d['rppa_levels'], 
     d['alterations'],
     d['microbiome']])

In [99]:
# write indices to CSV
pd.DataFrame(indices, columns=['index']).to_csv("DaniDatasets/indices_for_preprocessed.csv", index=False)

In [100]:
final_df = final_df.rename(columns={'OS_STATUS':'event', 'OS_MONTHS': 'time'})

In [101]:
# write df to CSV
final_df.to_csv("DaniDatasets/preprocessed.csv")

In [104]:
final_df.shape

(319, 8378)

In [103]:
indices

[0, 4, 2001, 2489, 3190, 5466, 5589, 7038, 8378]

In [105]:
low_var_columns = ['metilation_AXIN2', 'metilation_EPHB2', 'metilation_SERTAD1', 'alterations_ALK_TPM1', 'alterations_BCKDHB_TTK', 'alterations_BRIP1_RHBDF2', 'alterations_BRIP1_ST6GALNAC2', 'alterations_CIDECP_FANCD2', 'alterations_CNOT11_NPAS2', 'alterations_ENTPD5_NUBPL', 'alterations_GUCY1A3_HPGD', 'alterations_RUNX1_WDR4', 'alterations_SREBF1_TOM1L2', 'alterations_TOP3A_ZCCHC8', 'alterations_APITD1', 'alterations_MIR148A', 'alterations_MPST', 'alterations_RBM38', 'alterations_SIRT6']

In [107]:
# final_df[low_var_columns].std().max()